In [ ]:
import sys
sys.path.insert(0, '/nfs/general/shared/code/rpi_color_detector/classifier/')
import image_provider
import tensorflow as tf

In [9]:
def TrainingDataGen():
    ip = image_provider.ImageProvider()
    d = ip.listImages('classified')
    validClassNames = sorted(
        [x for x in d if x not in ['unclassified', '_garbage']])
    print(validClassNames)
    for i, className in enumerate(validClassNames):
        for image_path in d[className]:
            yield ip.filePath(image_path), i

def LoadImage(filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image.set_shape([480, 640, 3])
    return image

def ExtractRoiForPrediction(image):
    x=0.3
    target_width=int(640*x)
    target_height=int(480*x)
    offset_width=(640-target_width)//2
    offset_height=(480-target_height)//2
    return tf.image.crop_to_bounding_box(image, offset_height, offset_width, target_height, target_width)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomTranslation(0.05, 0.05)
    ]
)

AUTOTUNE = tf.data.AUTOTUNE

training_data = tf.data.Dataset.from_generator(TrainingDataGen, output_signature=(
    tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.int32)))
training_data = training_data.map(lambda imgName, label : (LoadImage(imgName), label), deterministic=False, num_parallel_calls=AUTOTUNE)
training_data = training_data.cache()
training_data = training_data.repeat()
training_data = training_data.shuffle(buffer_size=1000)
training_data = training_data.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
training_data = training_data.map(lambda x, y: (ExtractRoiForPrediction(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
training_data = training_data.batch(64)
training_data = training_data.prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (5, 5), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7)
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
starting_point = tf.saved_model.load('/nfs/general/shared/tf_models/color_classifier')

print(starting_point.variables)
print(model.variables)

tb_callback = tf.keras.callbacks.TensorBoard(
    '/tmp/logs', update_freq=1, histogram_freq=1, write_images=True)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

model.fit(training_data, epochs=400, steps_per_epoch=32, callbacks=[tb_callback, model_checkpoint_callback])

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier3')
converter = tf.lite.TFLiteConverter.from_saved_model('/nfs/general/shared/tf_models/color_classifier') # path to the SavedModel directory
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

AttributeError: '_UserObject' object has no attribute 'fit'